# Tarefa 5: Usar a transformação em lote para obter inferências de um conjunto de dados grande

## Tarefa 5.1: Configuração do ambiente

Instale os pacotes e as dependências.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import time
from sagemaker.session import Session
from sagemaker.transformer import Transformer

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
sagemaker_session = sagemaker.Session()
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

Salve o modelo do laboratório de treinamento e ajuste no bucket-padrão do Amazon Simple Storage Service (Amazon S3). Configure um modelo usando **create_model** e configure **ModelDataUrl** para referenciar o modelo treinado.

In [ ]:
#set-up-model
# Upload the model and dataset to your Amazon S3 bucket
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

Carregue os registros em lote no bucket padrão do Amazon S3.

In [ ]:
#upload-dataset
s3_client.upload_file(Filename="batch_data.csv", Bucket=bucket, Key=f"{prefix}/batch_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/batch_data.csv"

### Tarefa 5.2: Criar um trabalho de transformação em lote

A transformação em lote gerencia automaticamente o processamento de conjuntos de dados grandes dentro dos limites dos parâmetros especificados. Quando um trabalho de transformação em lote é iniciado, o SageMaker inicializa instâncias de computação e distribui a carga de trabalho de inferência ou de pré-processamento entre elas. A transformação em lote particiona os objetos do Amazon S3 na entrada por chave e mapeia os objetos do Amazon S3 para as instâncias.

Use a transformação em lote quando precisar obter inferências de conjuntos de dados grandes ou quando não precisar de um endpoint persistente.

Para criar um trabalho de transformação em lote, defina as seguintes opções:
- **model_name**: o nome do modelo.
- **instance_type**: o tipo de instância do Amazon Elastic Compute Cloud (Amazon EC2) a ser usado, por exemplo, "ml.c4.xlarge".
- **instance_count**: o número de instâncias do EC2 a serem usadas.
- **assemble_with**: a maneira em que a saída é montada. Os valores válidos são "Line" ou "None".
- **strategy**: a estratégia usada para decidir como executar os registros em lote em uma só solicitação. Os valores válidos são "MultiRecord" e "SingleRecord".
- **accept**: o tipo de arquivo a ser aceito.
- **output_path**: o local do Amazon S3 para salvar o resultado da transformação. Se o valor não for especificado, os resultados serão armazenados em um bucket padrão.

In [ ]:
#create-batch-transformer
transformer = Transformer(
    model_name=model_name,
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

Use o conjunto de dados de teste como os registros de clientes e execute o trabalho de transformação em lote. O trabalho pode exigir até dez minutos para ser executado com esse conjunto de registros de clientes.

Confira [Usar a transformação em lote](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) para saber mais sobre os trabalhos de transformação em lote.

In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

## Tarefa 5.3: Visualizar os dados de previsão no Amazon S3

O trabalho de transformação em lote armazena o bucket e a pasta que você especificou quando configurou o transformador. É possível visualizar os resultados da previsão no Amazon S3, no console de gerenciamento da AWS ou no notebook.

Se você quiser baixar e visualizar a saída do console, navegue até o Amazon S3, abra o bucket que começa com **sagemaker-** e navegue até o objeto localizado em **/sagemaker/mlasms/batch-transform/test**. Baixe o objeto **batch_data.csv.out** e abra-o com um editor de bloco de notas. O arquivo contém centenas de valores previstos para os registros de clientes que foram executados por meio do trabalho de transformação em lote.

Um exemplo da saída também pode ser exibido no notebook.


In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
!head batch_data.csv.out

### Conclusão

Parabéns! Você usou o Amazon SageMaker para executar um trabalho de transformação em lote com sucesso.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.